In [1]:
import numpy as np
from pyrep import PyRep
from arm import CtRobot
from rigid_kinematics import dh_robot_config
import sympy as sp
import os
import transforms3d as t3d

pi = np.pi/2
pr = PyRep()

In [54]:
pr.launch(os.getcwd() + '/DH_ct.ttt', headless=True)

In [55]:
pr.start()
ct_robot = CtRobot()

## My calibration (based on vrep 'reference_frame')

In [58]:
D = []
theta = []
alpha = []
a = []
jointType = ['p', 'p', 'r', 'r', 'r', 'r', 'p']
for i in range(ct_robot._num_joints):
    tmp = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])
    if tmp[0] < -1e-6 or tmp[0] > 1e-6:
        alpha.append(tmp[0])
    else:
        alpha.append(0)
    if tmp[2] < -1e-6 or tmp[2] > 1e-6:
        theta.append(tmp[2])
    else:
        theta.append(0)

ai, aj, ak = ct_robot.DH_frames[0].get_orientation()

In [60]:
theta

[0, 0, 0, 1.5708118677139282, 0, -1.5707964897155762, 0]

In [21]:
ct_robot.DH_frames[0].get_quaternion()

[0.5, -0.5, 0.5, 0.5]

## Based on Paper ( for convenience)

In [25]:
num_joints = 7
a = [0, 0, 0, 0, 0.08, 0.08, 0.0557, 0]
alpha = [pi/2, -pi/2, 0, pi/2, pi/2, pi/2, -pi/2, 0]
D = [0, 0, 0, 0, 0, 0, 0.0274, 0.115]
theta = [0, 0, 0, pi/2, 0, -pi/2, 0, pi/2] 
jointType = ['p', 'p', 'r', 'r', 'r', 'r', 'p']
ai = pi/2
aj = 0
ak = pi/2

In [26]:
robot = dh_robot_config(num_joints, alpha, theta, D, a, jointType, ai, aj, ak)

## V-REP

In [31]:
joint_pos = [ct_robot.joints[i].get_joint_position() for i in range(ct_robot._num_joints)]
print('Joint position is', joint_pos)
print('Tip position is', ct_robot._ik_tip.get_position())
print('Tip orientation is', ct_robot._ik_tip.get_orientation())

Joint position is [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Tip position is [0.0010542183881625533, -0.6585248708724976, 0.22177208960056305]
Tip orientation is [1.1082737216838723e-07, -4.470347647611561e-08, 8.568166975919667e-08]


## DH-FK

In [33]:
T_j7 = sp.lambdify(robot.q, robot._T[6])(*(0, 0,0,0,0,0,0))
pos_j7 = T_j7[:3, 3]
R_j7 = T_j7[:3, :3]
ori_j7 = t3d.euler.mat2euler(R_j7)
print('Tip position is', pos_j7)
print('Tip orientation is', ori_j7)

Tip position is [0.0171875  0.18787307 0.0456182 ]
Tip orientation is (2.8860972799412714, 0.14697519066350537, 1.8262917004434172)


num_joints = 8
alpha = [-pi/2, pi/2, 0, pi/2, pi/2, -pi/2, -pi/2, 0]
theta = [0, pi/2, 0, pi/2, pi/2, 0, -pi/2, 0]
D = [0, 0, 0, 0, 0, 0, 0, 0.05]
a = [0, 0, 0, 0, 0.08, 0.08, 0, 0]
jointType = ['p', 'p', 'p', 'r', 'r', 'r', 'r', 'p']
ai = pi/2
aj = 0
ak = 0

robot1 = dh_robot_config(num_joints, alpha, theta, D, a, jointType, ai, aj, ak)

In [ ]:
robot._Tjoint[1]

## Compare

In [ ]:
for i in range(ct_robot._num_joints):
    print('Tip position is', ct_robot._ik_tip.get_position())
    print('Tip orientation is', ct_robot._ik_tip.get_orientation())

In [53]:
pr.shutdown()
